## Fecha de procesamiento de AFIP

In [1]:
Fechas = ['11/11/2024', '12/11/2024', '13/11/2024']

## Paquetes

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from datetime import datetime
import calendar
import locale

## Cargar programas

In [3]:
%run "AFIP_0_Payway.ipynb"
%run "AFIP_2_Funciones.ipynb"
%run "AFIP_3_Listas.ipynb"

## Variables

In [4]:
Ruta_Descarga_Payway = 'C:/Users/tomas/Downloads/Movimientos En Linea Delimitado por comas.csv'
Ruta_Nueva_Payway = 'J:/My Drive/Forraje/AFIP' 
Nombre_Nuevo = 'Payway.csv'

In [5]:
Email_Payway = 'carolina8101924@gmail.com'
Contraseña_Payway = '123Nogue$'

In [6]:
Usuario_AFIP = '27202147025'
Contraseña_AFIP = '123Carolina$'

In [7]:
Empresa = 'MARQUEZ CAROLINA MARIEL'
Punto_de_Ventas = '00002-Las Piedras 2837 - Kilometro 45, Buenos Aires'

In [8]:
Tipo_Comprobante = 'Factura C'
Tipo_Concepto = 'Productos'
Tipo_Condicion_IVA = 'Consumidor Final'

In [9]:
Tipo_Comprobante_Value = Tipos_de_Comprobantes.get(Tipo_Comprobante)
Tipo_Concepto_Value = Tipos_de_Conceptos.get(Tipo_Concepto)
Tipo_Condicion_IVA_Value = Tipos_Condicion_IVA.get(Tipo_Condicion_IVA)

## Payway

In [10]:
# Abrir pestaña.
Driver = Chrome()

# Descargar archivo de Payway.
Payway = Payway(Driver, Email_Payway, Contraseña_Payway)

# Esperar descarga.
Esperar_Descarga("C:/Users/tomas/Downloads/Movimientos En Linea Delimitado por comas.csv")

## Cambiar y procesar Excel con precios

In [11]:
# Cambiar ubicación.
Mover_y_Cambiar_Nombre_Archivo(Ruta_Descarga_Payway, Ruta_Nueva_Payway, Nombre_Nuevo)

# Convertir archivo de Payway a Excel.
%run "AFIP_1_Excel.ipynb"

## Dataframe

In [12]:
# Ruta del archivo con los precios a subir.
Archivo = f"J:/My Drive/Forraje/AFIP/AFIP.xlsx"

# Crear DataFrame.
df = pd.read_excel(Archivo)

In [13]:
# Filtrar df con las fechas especificadas al comienzo.
df = df[df['Fecha'].isin(Fechas)]

In [14]:
# Resetear el índice del DataFrame.
df = df.reset_index(drop=True)

In [15]:
Fecha = df['Fecha']
Descripcion = df['Descripción']
Precio = df['Precio']

## AFIP

In [16]:
# Abrir pestaña.
Driver = Chrome()

# Ingreso a AFIP.
Ingreso = Paso_0(Driver, Usuario_AFIP, Contraseña_AFIP)

# Bucle de AFIP.
for i in range (0, len(Descripcion)):
    Pestaña_1 = Paso_1(Driver)
    Pestaña_2 = Paso_2(Driver, Punto_de_Ventas, Tipo_Comprobante_Value)
    Pestaña_3 = Paso_3(Driver, Fecha[i], Tipo_Concepto_Value)
    Pestaña_4 = Paso_4(Driver, Tipo_Condicion_IVA_Value)
    Pestaña_5 = Paso_5(Driver, Descripcion[i], Precio[i])
    Pestaña_6 = Paso_6(Driver)

## Estadística

In [17]:
# Ruta del archivo con los ingresos por día.
Ingresos = f"J:/My Drive/Forraje/Ingresos.xlsx"

# Crear DataFrame.
df_Ingresos = pd.read_excel(Ingresos)

In [18]:
# Facturaciones de los días buscados.
for i in range(0, len(Fechas)):
    
    # Filtrar el df con la fecha actual.
    df_Facturacion = df[df['Fecha'] == Fechas[i]]
    
    # Facturación del día.
    Facturacion = df_Facturacion['Precio'].sum()
    
    # Dato para agregar al df.
    Dato = {'Fecha': Fechas[i], 'Ingreso': Facturacion}
    df_Dato = pd.DataFrame([Dato])
    
    # Concatenar el nuevo DataFrame con el DataFrame original, colocando el nuevo dato al principio
    df_Ingresos = pd.concat([df_Dato, df_Ingresos], ignore_index=True)

In [19]:
# Guardar el DataFrame en un archivo Excel
df_Ingresos.to_excel(Ingresos, index=False, sheet_name='Estadística')

## Facturación

In [20]:
# Convertir la columna 'Fecha' a tipo datetime
df_Ingresos['Fecha'] = pd.to_datetime(df_Ingresos['Fecha'], format="%d/%m/%Y")

In [21]:
# Establecer la localización a español.
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Mes actual
Mes = datetime.now().month

In [22]:
# Filtrar el DataFrame para solo los datos del mes actual
df_Mes = df_Ingresos[df_Ingresos['Fecha'].dt.month == Mes]

In [23]:
# Facturación del mes actual hasta ahora.
Facturacion_Mensual = df_Mes['Ingreso'].sum()

## Total

In [24]:
print('Facturación total del mes de', calendar.month_name[Mes], 'hasta ahora:', "${:,.0f}".format(Facturacion_Mensual).replace(',', '.'))

Facturación total del mes de noviembre hasta ahora: $2.509.852
